In [60]:
import os
import pickle
import torch
import numpy as np
import IPython.display as ipd
import torch.nn as nn

In [2]:
from melgan.interface import *

In [3]:
from factory.AutoVC import AutoVC
from factory.MetaConv import MetaConv

In [4]:
ROOT = "train_data"

In [5]:
erroment_uttr_idx = 16
max_uttr_idx = 60
num_speaker = 40

In [6]:
LEN_CROP = 176
DIM_NECK = 44
DIM_EMB = 256
D_MODEL = 80
FREQ = 22
BATCH_SIZE = 2
DEVICE = device = "cuda:0"

In [7]:
org = AutoVC(DIM_NECK,DIM_EMB,2*DIM_EMB,FREQ).eval().to(DEVICE)
org.load_state_dict(torch.load("model/autovc_pytorch_org_en.pt", map_location=DEVICE))
org_dc = AutoVC(DIM_NECK,DIM_EMB,2*DIM_EMB,FREQ).eval().to(DEVICE)
org_dc.load_state_dict(torch.load("model/autovc_pytorch_org_dc_en.pt", map_location=DEVICE))
conv = MetaConv(44,256,512,22).to(DEVICE) 
conv.load_state_dict(torch.load("model/autovc_pytorch_meta_conv_final_en.pt", map_location=DEVICE))

<All keys matched successfully>

In [8]:
vocoder = MelVocoder(model_name='model/static/multi_speaker')

In [9]:
all_speaker = sorted(next(iter(os.walk(f'{ROOT}')))[1][:num_speaker])
all_speaker

['p225',
 'p226',
 'p227',
 'p228',
 'p229',
 'p230',
 'p231',
 'p232',
 'p233',
 'p234',
 'p236',
 'p237',
 'p238',
 'p239',
 'p240',
 'p241',
 'p243',
 'p244',
 'p245',
 'p246',
 'p247',
 'p248',
 'p249',
 'p250',
 'p251',
 'p252',
 'p253',
 'p254',
 'p255',
 'p256',
 'p257',
 'p258',
 'p259',
 'p260',
 'p261',
 'p262',
 'p263',
 'p264',
 'p265',
 'p266']

In [10]:
metadata = pickle.load(open('train_data/train.pkl', "rb"))[:num_speaker]
metadata_copy = []
for data in metadata:
    if data[0] in all_speaker:
        metadata_copy.append(data)
metadata = sorted(metadata_copy)
for data in metadata:
    print(data[0])

p225
p226
p227
p228
p229
p230
p231
p232
p233
p234
p236
p237
p238
p239
p240
p241
p243
p244
p245
p246
p247
p248
p249
p250
p251
p252
p253
p254
p255
p256
p257
p258
p259
p260
p261
p262
p263
p264
p265
p266


In [46]:
source = 0
target = 0
sound = 3

In [47]:
metadata[source][0]
emb_org = np.expand_dims(metadata[source][1],axis=0)
emb_org.shape

(1, 256)

In [48]:
p_ = metadata[source][sound].replace("\\","/")
uttr_org = np.load(f"train_data/{p_}")[50:226]
uttr_org  = np.expand_dims(uttr_org ,axis=0)
uttr_org.shape

(1, 176, 80)

In [49]:
metadata[target][0]
emb_trg =  np.expand_dims(metadata[target][1],axis=0)
emb_trg.shape

(1, 256)

## 原始聲音

In [50]:
audio_s = vocoder.inverse(torch.from_numpy(np.transpose(uttr_org,(0,2,1))))
ipd.Audio(audio_s,rate = 22050)

## 目標聲音

In [51]:
p_ = metadata[target][sound].replace("\\","/")
uttr_target = np.expand_dims(np.load(f"train_data/{p_}")[50:226].T,axis=0)
audio_s = vocoder.inverse(torch.from_numpy(uttr_target))
ipd.Audio(audio_s,rate = 22050)

In [52]:
_,x_org,_ = org(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))
_,x_org_dc,_ = org_dc(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))
_,x_conv,_ = conv(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))

In [53]:
uttr_org = x_org[0,  :, :]
uttr_org_dc = x_org_dc[0,  :, :]
uttr_conv = x_conv[0,  :, :]

In [54]:
audio_org = vocoder.inverse(uttr_org.transpose(1,2))
audio_org_dc = vocoder.inverse(uttr_org_dc.transpose(1,2))
audio_conv = vocoder.inverse(uttr_conv.transpose(1,2))

## 轉換聲音

In [55]:
ipd.Audio(audio_org,rate = 22050)

In [56]:
ipd.Audio(audio_org_dc,rate = 22050)

In [22]:
ipd.Audio(audio_conv,rate = 22050)

## Evaluate

In [23]:
# 評估用
C = torch.load('model/static/metadv_en_openslr.pt').to(DEVICE)

In [24]:
def get_trans_mel(model,sample,emb_org,emb_trg):
    return model(sample.to(device),emb_org.to(device),emb_trg.to(device))[1][0,:,:].detach().cpu()

In [25]:
def get_crop_mel(tmp,len_crop=176):
    if tmp.shape[0] < len_crop:
        pad_size = int(len_crop - tmp.shape[0])
        npad = [(0, 0)] * tmp.ndim
        npad[0] = (0, pad_size)
        tmp = np.pad(tmp, pad_width=npad, mode='constant', constant_values=0)
        melsp = torch.from_numpy(tmp)
            
    elif tmp.shape[0] == len_crop:
        melsp = torch.from_numpy(tmp)
    else:
        left = np.random.randint(0, tmp.shape[0] - len_crop)
        melsp = torch.from_numpy(tmp[left : left + len_crop, :])
    return melsp.to(device)

In [26]:
import torch.nn.functional as F
import random
def get_dv(speaker):
    all_uttr = next(iter(os.walk(f'{ROOT}/{speaker}')))[2][:erroment_uttr_idx]
    _dv = torch.zeros((1,256))
    for uttr in all_uttr:
        mel = torch.from_numpy(np.load(f'{ROOT}/{speaker}/{uttr}')).unsqueeze(0).to(device)
        if mel.shape[1] < 176:
            pad_size = int(176 - mel.shape[1])
            mel = F.pad(mel,(0,0,int(pad_size/2),int(pad_size/2)),'constant',0)
            print(mel.shape)
        _dv += C(mel)[1].detach().cpu()
    _dv = _dv/(erroment_uttr_idx+1)
    random_uttr = next(iter(os.walk(f'{ROOT}/{speaker}')))[2][random.randint(erroment_uttr_idx,max_uttr_idx)]
    mel = get_crop_mel(np.load(f'{ROOT}/{speaker}/{random_uttr}')).unsqueeze(0)
    emb_org = _dv.to(device) 
    return mel,emb_org,speaker

In [27]:
all_dv = []
for i,speaker in enumerate(all_speaker):
    all_dv.append(get_dv(speaker))

torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])


In [28]:
def get_cos_rc(cos_res):
    rc = 0.0
    rc_ = 0.0
    N = len(cos_res)
    for i in range(N):
        total = sum(cos_res[i][i])
        others = (total - cos_res[i][i][i])/(N-1)
        rc_ += others
        rc += cos_res[i][i][i]
    return rc/N,rc_/N

In [29]:
def get_cos_trans(cos_res):
    trans = []
    trans_ = []
    N = len(cos_res)
    for i in range(N):
        trans_cos = np.sum(np.diagonal(cos_res[i])) - cos_res[i][i][i]
        trans.append(trans_cos / (N-1))
        trans_.append(  (np.sum(cos_res[i]) - np.sum(np.diagonal(cos_res[i]))) / ((N-1)*N))
    
    return sum(trans)/len(trans),sum(trans_)/len(trans_)

In [80]:
cos_reslut_rc_org = np.zeros((num_speaker,num_speaker,num_speaker))
cos_reslut_rc_dc = np.zeros((num_speaker,num_speaker,num_speaker))
cos_reslut_rc_conv = np.zeros((num_speaker,num_speaker,num_speaker)) 
for i,data in enumerate(metadata):
    sp_s = data[0]
    source_mel = get_crop_mel(np.load(f"{ROOT}//{data[random.randint(erroment_uttr_idx,max_uttr_idx)]}")).unsqueeze(0)
    emb_org = torch.from_numpy(data[1]).unsqueeze(0)
    print(f"Now Processing --- {sp_s}")
    for j,data in enumerate(metadata):
        sp_o = data[0]
        emb_trg = torch.from_numpy(data[1]).unsqueeze(0)
        trans_mel_org = get_trans_mel(org,source_mel,emb_org,emb_trg).to(device)
        trans_mel_dc = get_trans_mel(org_dc,source_mel,emb_org,emb_trg).to(device)
        trans_mel_conv = get_trans_mel(conv,source_mel,emb_org,emb_trg).to(device)
        org_dv = C(trans_mel_org)[1]
        dc_dv = C(trans_mel_dc)[1]
        conv_dv = C(trans_mel_conv)[1]
   
        if i==j:
            print(f"Reconstruct ---- {sp_s} to {sp_o}")
        else:
            print(f"Trans --- {sp_s} to {sp_o}")
        
        for k,data in enumerate(all_dv):
            _,emb,_ = data
            cos = torch.clamp(nn.functional.cosine_similarity(org_dv,emb, dim=1, eps=1e-8),min=0.0).detach().cpu().numpy()[0].astype(np.float32)
            cos_reslut_rc_org[i][j][k] = cos
            cos = torch.clamp(nn.functional.cosine_similarity(dc_dv,emb, dim=1, eps=1e-8),min=0.0).detach().cpu().numpy()[0].astype(np.float32)
            cos_reslut_rc_dc[i][j][k] = cos
            cos = torch.clamp(nn.functional.cosine_similarity(conv_dv,emb, dim=1, eps=1e-8),min=0.0).detach().cpu().numpy()[0].astype(np.float32)
            cos_reslut_rc_conv[i][j][k] = cos
            

Now Processing --- p225
Reconstruct ---- p225 to p225
Trans --- p225 to p226
Trans --- p225 to p227
Trans --- p225 to p228
Trans --- p225 to p229
Trans --- p225 to p230
Trans --- p225 to p231
Trans --- p225 to p232
Trans --- p225 to p233
Trans --- p225 to p234
Trans --- p225 to p236
Trans --- p225 to p237
Trans --- p225 to p238
Trans --- p225 to p239
Trans --- p225 to p240
Trans --- p225 to p241
Trans --- p225 to p243
Trans --- p225 to p244
Trans --- p225 to p245
Trans --- p225 to p246
Trans --- p225 to p247
Trans --- p225 to p248
Trans --- p225 to p249
Trans --- p225 to p250
Trans --- p225 to p251
Trans --- p225 to p252
Trans --- p225 to p253
Trans --- p225 to p254
Trans --- p225 to p255
Trans --- p225 to p256
Trans --- p225 to p257
Trans --- p225 to p258
Trans --- p225 to p259
Trans --- p225 to p260
Trans --- p225 to p261
Trans --- p225 to p262
Trans --- p225 to p263
Trans --- p225 to p264
Trans --- p225 to p265
Trans --- p225 to p266
Now Processing --- p226
Trans --- p226 to p225
Re

## Real Data

In [71]:
cos_gt = np.zeros((num_speaker,num_speaker))
for i,speaker in enumerate(all_speaker):
    mel,_,_ = get_dv(speaker)
    _dv = C(mel)[1].detach().cpu()
    for j,data in enumerate(all_dv):
        dv = data[1].detach().cpu()
        cos = torch.clamp(nn.functional.cosine_similarity(_dv,dv, dim=1, eps=1e-8),min=0.0).detach().cpu().numpy()[0].astype(np.float32)
        cos_gt[i][j] = cos
np.diagonal(cos_gt).mean(), (np.sum(cos_gt) - (np.sum(np.diagonal(cos_gt)))) /(num_speaker*(num_speaker-1))

torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])


(0.9219933703541756, 0.7286213262914083)

In [81]:
get_cos_rc(cos_reslut_rc_org)

(0.922068253159523, 0.7408166933136109)

In [82]:
get_cos_rc(cos_reslut_rc_dc)

(0.9330819979310035, 0.7447800768682591)

In [83]:
get_cos_rc(cos_reslut_rc_conv)

(0.9329326301813126, 0.7502661349299626)

In [84]:
get_cos_trans(cos_reslut_rc_org)

(0.8532403291417999, 0.7677591242899114)

In [85]:
get_cos_trans(cos_reslut_rc_dc)

(0.7684353037140309, 0.762184486185702)

In [86]:
get_cos_trans(cos_reslut_rc_conv)

(0.8876791428679075, 0.7675299781728058)

In [87]:
rc_reslut_rc_orgs = [0.922]
rc_reslut_rc_orgdcs = [0.923]
rc_reslut_rc_convss = [0.921]

_rc_reslut_rc_orgs = [0.764]
_rc_reslut_rc_orgdcs = [0.743]
_rc_reslut_rc_convss = [0.759]

trans_reslut_rc_orgs = [0.853,0.854,0.853]
trans_reslut_rc_orgdcs = [0.768,0.77]
trans_reslut_rc_convss = [0.888,0.892]


_trans_reslut_rc_orgs = [0.771]
_trans_reslut_rc_orgdcs = [0.762]
_trans_reslut_rc_convss = [0.77]

In [89]:
np.array(trans_reslut_rc_orgs).std()

0.00047140452079103207